In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import seaborn as sns
from scipy.sparse import vstack, hstack, csr_matrix, save_npz, load_npz
sns.set()
import gc
from sklearn.preprocessing import MinMaxScaler

from utils.schemas import *
from utils.functions import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df_imp = pd.read_csv('docs/ft_importances_20190831.csv')

In [3]:
df_imp.head(10)

,feature,importance
0,R1,2.415106
1,card1,1.695437
2,R12,1.606923
3,TransactionAmt,1.595195
4,N6,1.575507
5,C1,1.232615
6,N1,1.203357
7,N5,1.167223
8,card2,1.147620
9,R4,1.136447


In [4]:
# X_cols = df_imp[df_imp.importance > 0.1].feature.to_list()
X_cols = df_imp.feature[:100].to_list()

In [5]:
data_folder = 'input'

In [6]:
train = pd.read_csv(data_folder+'/train_ft_eng_1.csv', dtype = schema_ft_eng_1, usecols=X_cols)
test = pd.read_csv(data_folder+'/test_ft_eng_1.csv', dtype = schema_ft_eng_1, usecols=X_cols)

In [7]:
# train = csr_matrix(train)
# test = csr_matrix(test)
# gc.collect()

In [8]:
train.shape

(590540, 100)

In [9]:
train.head()

,C1,C10,C11,C12,C13,C14,C2,C4,C6,C7,...,R14,R17,R18,R19,R20,R21,R22,R27,R28,R29
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,1.414062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,0.000000,1.546875,1.499023,NaN,0.000000,NaN,NaN,NaN,NaN
3,2.0,0.0,1.0,0.0,25.0,1.0,5.0,0.0,4.0,0.0,...,NaN,0.568359,NaN,0.563965,0.0,0.483887,0.0,NaN,NaN,NaN
4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# test['isFraud'] = np.nan
train_index = train.shape[0]
test_index = test.shape[0]
df = pd.concat([train, test], axis=0)
df.shape

(1097231, 100)

In [11]:
del train
del test
gc.collect()

7

In [12]:
cat_ft_sel = list()
for c in df.columns:
    if c in cat_ft:
        cat_ft_sel.append(c)
cat_ft_sel[:4]

['DeviceInfo', 'M4', 'M5', 'M6']

In [13]:
num_cols = list()
for c in df.columns:
    if c not in cat_ft:
        num_cols.append(c)
num_cols[:5]

['C1', 'C10', 'C11', 'C12', 'C13']

In [14]:
mms = MinMaxScaler()

In [15]:
df[num_cols] = mms.fit_transform(df[num_cols])

In [16]:
df[num_cols[:5]].head()

,C1,C10,C11,C12,C13
0,0.000213,0.000000,0.000627,0.0,0.000343
1,0.000213,0.000000,0.000314,0.0,0.000343
2,0.000213,0.000000,0.000314,0.0,0.000343
3,0.000427,0.000000,0.000314,0.0,0.008568
4,0.000213,0.000307,0.000314,0.0,0.000343


In [17]:
df_csr_num = csr_matrix(df[num_cols])

In [18]:
df_csr_num.shape

(1097231, 68)

In [19]:
def del_col_same_value(df):
    new_sel_cols = list()
    for c in df.columns:
        if len(df[c].unique()) > 1:
            new_sel_cols.append(c)
    return df[new_sel_cols]

In [20]:
def from_df_to_sparse(df):
    df_dummies = csr_matrix(pd.DataFrame())
    for c in df.columns:
        if c in cat_ft and c not in ['isFraud', 'TransactionDT']:
            print('Transform col: {}'.format(c))
            try:
                dums = csr_matrix(del_col_same_value(pd.get_dummies(df[c], prefix=c, dummy_na=False)))
                df_dummies = hstack([df_dummies, dums])
#                 dums = csr_matrix(pd.get_dummies(df[c], prefix=c, dummy_na=False))
#                 df_dummies = hstack([df_dummies, dums])
            except:
                print('Error in {}'.format(c))
        else:
            pass
            
    return df_dummies

In [21]:
# def from_df_to_sparse(df):
    
#     df_dummies = csr_matrix(pd.DataFrame())
    
#     for c in df.columns:
#         print('Transform col: {}'.format(c))
#         if c in cat_ft and c not in ['isFraud', 'TransactionDT']:
# #             if len(df[c].unique()) > 1000:
# #                 try:
# #                     dums = csr_matrix(del_col_same_value(pd.get_dummies(pd.cut(df[c], 256), prefix=c, dummy_na=False)))
# #                     df_dummies = hstack([df_dummies, dums])
# #                 except:
# #                     print('Error in {}'.format(c))
# #             else:
#             try:
# #                 dums = csr_matrix(del_col_same_value(pd.get_dummies(df[c], prefix=c, dummy_na=False)))
# #                 df_dummies = hstack([df_dummies, dums])
#                 dums = csr_matrix(pd.get_dummies(df[c], prefix=c, dummy_na=False))
#                 df_dummies = hstack([df_dummies, dums])
#             except:
#                 print('Error in {}'.format(c))
#         elif c not in cat_ft + ['isFraud', 'TransactionDT']:
#             try:
#                 dums = csr_matrix(df[c].fillna(0))
#                 df_dummies = hstack([df_dummies, dums])
#             except:
#                 print('Error in {}'.format(c))
# #             if df[c].isna().sum() == 0:
# #                 dums = csr_matrix(pd.get_dummies(pd.cut(df[c], 256), prefix=c, dummy_na=False))
# #                 df_dummies = hstack([df_dummies, dums])
# #             else:
# #                 dums = csr_matrix(del_col_same_value(pd.get_dummies(pd.cut(df[c], 256), prefix=c, dummy_na=True)))
# #                 df_dummies = hstack([df_dummies, dums])
#         else:
#             pass
            
#     return df_dummies

In [22]:
df_csr_cat = from_df_to_sparse(df)
df_csr_cat.shape

Transform col: DeviceInfo
Transform col: M4
Transform col: M5
Transform col: M6
Transform col: P_emaildomain
Transform col: addr1
Transform col: card1
Error in card1
Transform col: card2
Transform col: card5
Transform col: card6
Transform col: id_19
Transform col: id_20
Transform col: id_31
Transform col: P_emaildomain_0
Transform col: R_emaildomain_0
Transform col: P_emaildomain_fe1
Transform col: card1_fe1
Transform col: card2_fe1
Transform col: card5_fe1
Transform col: card6_fe1
Transform col: addr1_fe1
Transform col: M6_fe1
Transform col: P_emaildomain_0_fe1
Transform col: P_emaildomain_fe2
Transform col: card1_fe2
Transform col: card2_fe2
Transform col: card5_fe2
Transform col: card6_fe2
Transform col: addr1_fe2
Transform col: M4_fe2
Transform col: M5_fe2
Transform col: P_emaildomain_0_fe2


(1097231, 6372)

In [23]:
df_final = hstack([df_csr_cat, df_csr_num])
df_final.shape

(1097231, 6440)

In [24]:
# df_final = from_df_to_sparse(df)
# df_final.shape

In [25]:
print('Saving train data...')
save_npz(data_folder+'/compressed_train_ft_eng_1.npz', df_final.tocsr()[:train_index], compressed=True)
print('Saving test data...')
save_npz(data_folder+'/compressed_test_ft_eng_1.npz',  df_final.tocsr()[train_index:],  compressed=True)
print('Completed!')

Saving train data...
Saving test data...
Completed!


In [26]:
df_final.tocsr()[:train_index].shape

(590540, 6440)

In [13]:
# #Transform data using small groups to reduce memory usage
# # m = 50000
# # train_sparse = vstack([csr_matrix(train[X_cols].iloc[i*m:(i+1)*m,:]) for i in range(train.shape[0] // m + 1)])
# print('Saving train data...')
# save_npz(data_folder+'/train.npz', df_final.tocsr()[:train_index], compressed=True)
# print('Saving test data...')
# save_npz(data_folder+'/test.npz',  df_final.tocsr()[train_index:],  compressed=True)
# print('Completed!')

Saving train data...
Saving test data...
Completed!


In [78]:
train_sparse.shape

(590540, 243)